# Linear Regressing Player Data
### Version 1: 23/08/2023

Using the data from @vaastav (big shout out) - linear regressing the points achieved in a game by a player, based on a selection of inputs that I should have available beforehand. I've got a strong hypothesis on the game outcome!

In [9]:
import pandas as pd

In [73]:
years = []
for i in range(0,4):
    j = str(i)
    k = str(i+1)
    years.append("202"+j+"-2"+k)
    print(years)

all_years = pd.DataFrame()
for year in years:
    url = "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/{}/gws/merged_gw.csv".format(year)
    df = pd.read_csv(url)
    all_years = all_years.append(df, ignore_index=True).fillna('')
    all_years['season'] = int(year[:4])

['2020-21']
['2020-21', '2021-22']
['2020-21', '2021-22', '2022-23']
['2020-21', '2021-22', '2022-23', '2023-24']


C:\Users\chris\AppData\Local\Temp\ipykernel_10144\1315968431.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_years = all_years.append(df, ignore_index=True).fillna('')
C:\Users\chris\AppData\Local\Temp\ipykernel_10144\1315968431.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_years = all_years.append(df, ignore_index=True).fillna('')
C:\Users\chris\AppData\Local\Temp\ipykernel_10144\1315968431.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_years = all_years.append(df, ignore_index=True).fillna('')
C:\Users\chris\AppData\Local\Temp\ipykernel_10144\1315968431.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
 

In [74]:
all_years.dtypes

name                           object
position                       object
team                           object
xP                            float64
assists                         int64
bonus                           int64
bps                             int64
clean_sheets                    int64
creativity                    float64
element                         int64
fixture                         int64
goals_conceded                  int64
goals_scored                    int64
ict_index                     float64
influence                     float64
kickoff_time                   object
minutes                         int64
opponent_team                   int64
own_goals                       int64
penalties_missed                int64
penalties_saved                 int64
red_cards                       int64
round                           int64
saves                           int64
selected                        int64
team_a_score                    int64
team_h_score

In [75]:
all_years.head()

,name,position,team,xP,assists,bonus,bps,clean_sheets,creativity,element,...,value,was_home,yellow_cards,GW,season,expected_assists,expected_goal_involvements,expected_goals,expected_goals_conceded,starts
0,Aaron Connolly,FWD,Brighton,0.5,0,0,-3,0,0.3,78,...,55,True,0,1,2023,,,,,
1,Aaron Cresswell,DEF,West Ham,2.1,0,0,11,0,11.2,435,...,50,True,0,1,2023,,,,,
2,Aaron Mooy,MID,Brighton,0.0,0,0,0,0,0.0,60,...,50,True,0,1,2023,,,,,
3,Aaron Ramsdale,GK,Sheffield Utd,2.5,0,0,12,0,0.0,483,...,50,True,0,1,2023,,,,,
4,Abdoulaye Doucouré,MID,Everton,1.3,0,0,20,1,44.6,512,...,55,False,0,1,2023,,,,,
